In [114]:
import sys
sys.path.append('..')

import numpy as np
from src.helpers import load_csv_data, standardize, remove_columns, predict_labels, create_csv_submission
from src.logistic.implementations import logistic_regression
from src.logistic.loss import compute_loss
from src.split import split_data

from src.logistic.sigmoid import sigmoid

%reload_ext autoreload
%autoreload 2

In [98]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv', sub_sample=True)

In [102]:
# Clean the data
x, kept_columns = remove_columns(x_raw)

In [108]:
kept_columns

array([ True,  True,  True,  True, False, False, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False,  True])

In [62]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [63]:
# Standardize x and add the 1s column
x, mean_x, std_x = standardize(x)
tx = np.c_[np.ones((y.shape[0], 1)), x]

In [64]:
# Split the data
tx_train, y_train, tx_test, y_test = split_data(tx, y, 0.8)

In [75]:
initial_w = np.zeros((tx.shape[1], 1))

w, tr_loss = logistic_regression(y, tx, initial_w, 100000, 1e-5)

Current iteration=0, loss=173.28679513998634
||d|| = 128.40074714372167
Current iteration=10000, loss=137.13024636740454
||d|| = 7.951316954571551
Current iteration=20000, loss=133.02245176736926
||d|| = 5.1155856634786385
Current iteration=30000, loss=131.15012879335947
||d|| = 3.630796670451024
Current iteration=40000, loss=130.155591126139
||d|| = 2.7205822118830634
Current iteration=50000, loss=129.5773832555929
||d|| = 2.113973904640028
Current iteration=60000, loss=129.2184827293122
||d|| = 1.691483207359708
Current iteration=70000, loss=128.9828478695989
||d|| = 1.3901984118841977
Current iteration=80000, loss=128.81975911615433
||d|| = 1.172382689420134
Current iteration=90000, loss=128.7010102634187
||d|| = 1.0132957597390493
loss=128.61032914595822


In [76]:
te_loss = compute_loss(y_test, tx_test, w)
print(te_loss)

22.46759478233665


In [105]:
y_sub, x_sub_raw, ids_sub = load_csv_data('../data/test.csv')

In [112]:
x_sub = x_sub_raw[:, kept_columns]
tx_sub = np.c_[np.ones((y_sub.shape[0], 1)), x_sub]
y_sub = predict_labels(w, tx_sub)

In [116]:
create_csv_submission(ids_sub, y_sub, '../submissions/10-20.15-53.csv')